# Module 1: Prepare datasets 
**This notebook generates and transforms datasets into a ML ready state to be ingested into SageMaker Feature Store.**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`


---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Generate Online Grocery shopping dataset](#Generate-Online-Grocery-shopping-dataset)
1. [Transform raw features into Machine Learning ready features](#Transform-raw-features-into-Machine-Learning-ready-features)

# Background

This notebook generates a relational set of files describing customers’ orders over time.

We have preset the notebook to generate a sample of 100,000 synthetic grocery orders from a total of 10,000 synthetically generated customers list. For each customer, the notebook generates between 1 to 10 of their orders, with products purchased in each order. The notebook also generates a timestamp on which the order was placed. The goal of the generated dataset and the example notebooks contained in this repository is to illustrate how to use **SageMaker Feature Store** to predict which products will be in a user’s next order and to enable a grocery retail store to revolutionize how consumers discover and purchase groceries online.

# Setup

##### Prerequisites

In [2]:
%%capture
!pip install faker
!pip install python-dateutil

### Create sub-directory structure for data files

Note: If you re-run this cell, you will get warnings from mkdir command that says 'File exists'; these warnings can be ignored

In [3]:
# create sub-directories
!mkdir ../data/test
!mkdir ../data/train
!mkdir ../data/transformed
!mkdir ../data/partitions
!mkdir ../data/validation

#### Imports 

In [4]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from datetime import datetime, timezone, date
from faker import Faker
import pandas as pd
import numpy as np
import hashlib
import logging
import random

Set locale and seed for reproducability 

In [5]:
faker = Faker()
faker.seed_locale('en_US', 0)

In [6]:
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

In [7]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [8]:
logger.info(f'Using Pandas version: {pd.__version__}')

Using Pandas version: 1.0.1


#### Helper functions  

In [9]:
def generate_timestamp(start, end) -> str:
    start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
    timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
    return timestamp

In [10]:
def generate_date(start, end) -> str:
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    date = faker.date_between_dates(date_start=start, date_end=end).strftime('%Y-%m-%d')
    return date

In [11]:
def get_md5_hash(string: str) -> str:
    hash_object = hashlib.md5(string.encode())
    return hash_object.hexdigest()

In [12]:
def generate_event_timestamp():
    # naive datetime representing local time
    naive_dt = datetime.now()
    # take timezone into account
    aware_dt = naive_dt.astimezone()
    # time in UTC
    utc_dt = aware_dt.astimezone(timezone.utc)
    # transform to ISO-8601 format
    event_time = utc_dt.isoformat(timespec='milliseconds')
    event_time = event_time.replace('+00:00', 'Z')
    return event_time

# Generate Online Grocery shopping dataset

### Generate random synthetic Customer profiles 

In [13]:
class Customer:
    def __init__(self):
        self.customer_id = None
        self.name = None
        self.sex = None
        self.state = None
        self.age = None
        self.is_married = None
        self.active_since = None
        self.event_time = None
        
    def as_dict(self):
        return {'customer_id': self.customer_id, 
                'name': self.name,
                'sex': self.sex, 
                'state': self.state, 
                'age': self.age, 
                'is_married': self.is_married, 
                'active_since': self.active_since,
                'event_time': self.event_time
               }

In [14]:
def generate_customer(i) -> Customer:
    customer = Customer()
    profile = faker.profile()
    customer.customer_id = f'C{i}'
    customer.name = profile['name'].lower()
    customer.sex = profile['sex']
    customer.state = faker.state().lower()
    customer.age = random.randint(18, 91)
    customer.is_married = faker.boolean()
    customer.active_since = generate_timestamp('2016-01-01 00:00:00', '2020-01-01 00:00:01')
    customer.event_time = generate_event_timestamp()
    return customer

In [15]:
customer = generate_customer(1)
customer.__dict__

{'customer_id': 'C1',
 'name': 'steven lucas',
 'sex': 'M',
 'state': 'illinois',
 'age': 24,
 'is_married': False,
 'active_since': '2016-05-16 22:44:31',
 'event_time': '2021-12-02T18:42:16.871Z'}

In [16]:
customers = []
customer_ids = []
n = 10000  # number of synthetic customers to generate
for i in range(n):
    customer = generate_customer(i+1)
    customers.append(customer)
    customer_ids.append(customer.customer_id)

In [17]:
customers_df = pd.DataFrame([customer.as_dict() for customer in customers])

In [18]:
customers_df.head(5)

,customer_id,name,sex,state,age,is_married,active_since,event_time
0,C1,justin gutierrez,M,alaska,52,True,2017-11-29 14:55:16,2021-12-02T18:42:17.582Z
1,C2,karen cross,F,idaho,29,True,2019-06-06 05:09:02,2021-12-02T18:42:17.584Z
2,C3,amy king,F,oklahoma,70,True,2019-06-08 05:06:18,2021-12-02T18:42:17.586Z
3,C4,nicole hartman,F,missouri,52,True,2016-06-13 05:47:10,2021-12-02T18:42:17.587Z
4,C5,jessica powers,F,minnesota,31,True,2018-12-08 05:42:44,2021-12-02T18:42:17.588Z


### Generate random synthetic purchase orders 

Load list of `products` raw data

In [19]:
products_df = pd.read_csv('../data/raw/product_category_mapping.csv')
products_df['product_name'] = products_df['product_name'].str.lower()
products_df['product_category'] = products_df['product_category'].str.lower()
products_df.head(5)

,product_name,product_category,product_id
0,chocolate sandwich cookies,cookies cakes,P1
1,all-seasons salt,spices seasonings,P2
2,robust golden unsweetened oolong tea,tea,P3
3,smart ones classic favorites mini rigatoni wit...,frozen meals,P4
4,pure coconut water with orange,juice nectars,P5


Add event timestamp to the feature records 

In [20]:
event_timestamps = [generate_event_timestamp() for _ in range(len(products_df))]
products_df['event_time'] = event_timestamps
products_df.head(5)

,product_name,product_category,product_id,event_time
0,chocolate sandwich cookies,cookies cakes,P1,2021-12-02T18:42:32.677Z
1,all-seasons salt,spices seasonings,P2,2021-12-02T18:42:32.677Z
2,robust golden unsweetened oolong tea,tea,P3,2021-12-02T18:42:32.677Z
3,smart ones classic favorites mini rigatoni wit...,frozen meals,P4,2021-12-02T18:42:32.677Z
4,pure coconut water with orange,juice nectars,P5,2021-12-02T18:42:32.677Z


In [21]:
product_ids = products_df['product_id'].tolist()

##### Generate purchase orders specific to customers 

In [22]:
class Order:
    def __init__(self):
        self.order_id = None
        self.customer_id = None
        self.product_id = None
        self.purchase_amount = None
        self.is_reordered = None 
        self.purchased_on = None
        self.event_time = None
        
    def as_dict(self):
        return {'order_id': self.order_id, 
                'customer_id': self.customer_id, 
                'product_id': self.product_id,
                'purchase_amount': self.purchase_amount,
                'is_reordered': self.is_reordered,
                'purchased_on': self.purchased_on, 
                'event_time': self.event_time}

In [23]:
def generate_order(i) -> Order:
    order = Order()
    order.order_id = f'O{i}'
    order.customer_id = random.choice(customer_ids)
    order.product_id = random.choice(product_ids)
    order.purchase_amount = random.randint(1, 101) + round(random.random(), 2)
    order.is_reordered = random.choice([1, 1, 0])  # assume chance of reordering is twice as that of not reordering
    order.purchased_on = generate_timestamp('2020-01-01 00:01:01', '2021-06-01 00:00:01')
    order.event_time = generate_event_timestamp()
    return order

In [24]:
order = generate_order(1)
order.__dict__

{'order_id': 'O1',
 'customer_id': 'C6445',
 'product_id': 'P12560',
 'purchase_amount': 24.77,
 'is_reordered': 1,
 'purchased_on': '2020-12-27 04:13:28',
 'event_time': '2021-12-02T18:42:37.246Z'}

In [25]:
orders = []
n = 100000  # number of synthetic orders to generate
for i in range(n):
    order = generate_order(i+1)
    orders.append(order)

In [26]:
orders_df = pd.DataFrame([order.as_dict() for order in orders])
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,O1,C5731,P16,93.26,1,2021-03-29 03:40:49,2021-12-02T18:42:38.444Z
1,O2,C3541,P12802,67.98,1,2020-02-20 11:06:21,2021-12-02T18:42:38.444Z
2,O3,C7402,P8320,64.59,1,2021-05-03 08:12:42,2021-12-02T18:42:38.444Z
3,O4,C7356,P5165,63.51,0,2020-12-05 11:31:54,2021-12-02T18:42:38.445Z
4,O5,C5806,P12940,6.37,1,2021-01-26 22:53:32,2021-12-02T18:42:38.445Z


In [27]:
orders_df.shape

(100000, 7)

#### Write generated customers, products and orders data to local directory

In [28]:
customers_df.to_csv('../data/raw/customers.csv', index=False)

In [29]:
products_df.to_csv('../data/raw/products.csv', index=False)

In [30]:
orders_df.to_csv('../data/raw/orders.csv', index=False)

# Transform raw features into Machine Learning ready features

#### I) Transform raw `customers` data

In [31]:
customers_df = pd.read_csv('../data/raw/customers.csv')
customers_df.head(5)

,customer_id,name,sex,state,age,is_married,active_since,event_time
0,C1,justin gutierrez,M,alaska,52,True,2017-11-29 14:55:16,2021-12-02T18:42:17.582Z
1,C2,karen cross,F,idaho,29,True,2019-06-06 05:09:02,2021-12-02T18:42:17.584Z
2,C3,amy king,F,oklahoma,70,True,2019-06-08 05:06:18,2021-12-02T18:42:17.586Z
3,C4,nicole hartman,F,missouri,52,True,2016-06-13 05:47:10,2021-12-02T18:42:17.587Z
4,C5,jessica powers,F,minnesota,31,True,2018-12-08 05:42:44,2021-12-02T18:42:17.588Z


In [32]:
label_encoder = LabelEncoder()
min_max_scaler = MinMaxScaler()

In [33]:
customers_df.drop('name', axis=1, inplace=True)
customers_df.drop('state', axis=1, inplace=True)

In [34]:
bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
customers_df['age_range'] = pd.cut(customers_df.age, bins, labels=labels, include_lowest=True)
customers_df = pd.concat([customers_df, pd.get_dummies(customers_df['age_range'], prefix='age')], axis=1)
customers_df.drop('age', axis=1, inplace=True)
customers_df.drop('age_range', axis=1, inplace=True)

In [35]:
customers_df['sex'] = label_encoder.fit_transform(customers_df['sex'])
customers_df['is_married'] = label_encoder.fit_transform(customers_df['is_married'])

In [36]:
customers_df.head()

,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,1,1,2017-11-29 14:55:16,2021-12-02T18:42:17.582Z,0,0,0,1,0,0
1,C2,0,1,2019-06-06 05:09:02,2021-12-02T18:42:17.584Z,1,0,0,0,0,0
2,C3,0,1,2019-06-08 05:06:18,2021-12-02T18:42:17.586Z,0,0,0,0,1,0
3,C4,0,1,2016-06-13 05:47:10,2021-12-02T18:42:17.587Z,0,0,0,1,0,0
4,C5,0,1,2018-12-08 05:42:44,2021-12-02T18:42:17.588Z,0,1,0,0,0,0


In [37]:
customers_df['active_since'] =  pd.to_datetime(customers_df['active_since'], format='%Y-%m-%d %H:%M:%S')

In [38]:
def get_delta_in_days(date_time) -> int:
    today = date.today()
    delta = today - date_time.date()
    return delta.days

In [39]:
customers_df['n_days_active'] = customers_df['active_since'].apply(lambda x: get_delta_in_days(x))
customers_df['n_days_active'] = min_max_scaler.fit_transform(customers_df[['n_days_active']])
customers_df.drop('active_since', axis=1, inplace=True)

In [40]:
customers_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,1,1,2021-12-02T18:42:17.582Z,0,0,0,1,0,0,0.521918
1,C2,0,1,2021-12-02T18:42:17.584Z,1,0,0,0,0,0,0.142466
2,C3,0,1,2021-12-02T18:42:17.586Z,0,0,0,0,1,0,0.141096
3,C4,0,1,2021-12-02T18:42:17.587Z,0,0,0,1,0,0,0.887671
4,C5,0,1,2021-12-02T18:42:17.588Z,0,1,0,0,0,0,0.265753


In [41]:
customers_df.to_csv('../data/transformed/customers.csv', index=False)

#### II) Transform raw `products` data

In [42]:
products_df = pd.read_csv('../data/raw/products.csv')
products_df.head(5)

,product_name,product_category,product_id,event_time
0,chocolate sandwich cookies,cookies cakes,P1,2021-12-02T18:42:32.677Z
1,all-seasons salt,spices seasonings,P2,2021-12-02T18:42:32.677Z
2,robust golden unsweetened oolong tea,tea,P3,2021-12-02T18:42:32.677Z
3,smart ones classic favorites mini rigatoni wit...,frozen meals,P4,2021-12-02T18:42:32.677Z
4,pure coconut water with orange,juice nectars,P5,2021-12-02T18:42:32.677Z


In [43]:
products_df.drop('product_name', axis=1, inplace=True)

In [44]:
products_df = pd.concat([products_df, pd.get_dummies(products_df['product_category'], prefix='category')], axis=1)

In [45]:
products_df.drop('product_category', axis=1, inplace=True)
products_df.columns = products_df.columns.str.replace(' ', '_')

In [46]:
products_df.head(5)

,product_id,event_time,category_baby_food_formula,category_baking_ingredients,category_candy_chocolate,category_chips_pretzels,category_cleaning_products,category_coffee,category_cookies_cakes,category_crackers,...,category_hair_care,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt
0,P1,2021-12-02T18:42:32.677Z,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,P2,2021-12-02T18:42:32.677Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,P3,2021-12-02T18:42:32.677Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,P4,2021-12-02T18:42:32.677Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,P5,2021-12-02T18:42:32.677Z,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [47]:
products_df.to_csv('../data/transformed/products.csv', index=False)

#### III) Transform raw `orders` data

In [48]:
orders_df = pd.read_csv('../data/raw/orders.csv')
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,O1,C5731,P16,93.26,1,2021-03-29 03:40:49,2021-12-02T18:42:38.444Z
1,O2,C3541,P12802,67.98,1,2020-02-20 11:06:21,2021-12-02T18:42:38.444Z
2,O3,C7402,P8320,64.59,1,2021-05-03 08:12:42,2021-12-02T18:42:38.444Z
3,O4,C7356,P5165,63.51,0,2020-12-05 11:31:54,2021-12-02T18:42:38.445Z
4,O5,C5806,P12940,6.37,1,2021-01-26 22:53:32,2021-12-02T18:42:38.445Z


In [49]:
orders_df['purchased_on'] =  pd.to_datetime(orders_df['purchased_on'], format='%Y-%m-%d %H:%M:%S')
orders_df['n_days_since_last_purchase'] = orders_df['purchased_on'].apply(lambda x: get_delta_in_days(x))
orders_df['n_days_since_last_purchase'] = min_max_scaler.fit_transform(orders_df[['n_days_since_last_purchase']])


In [50]:
orders_df['purchase_amount'] = min_max_scaler.fit_transform(orders_df[['purchase_amount']])
orders_df['is_reordered'] = label_encoder.fit_transform(orders_df['is_reordered'])

In [51]:
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,O1,C5731,P16,0.913465,1,2021-03-29 03:40:49,2021-12-02T18:42:38.444Z,0.122093
1,O2,C3541,P12802,0.663168,1,2020-02-20 11:06:21,2021-12-02T18:42:38.444Z,0.903101
2,O3,C7402,P8320,0.629604,1,2021-05-03 08:12:42,2021-12-02T18:42:38.444Z,0.054264
3,O4,C7356,P5165,0.618911,0,2020-12-05 11:31:54,2021-12-02T18:42:38.445Z,0.343023
4,O5,C5806,P12940,0.053168,1,2021-01-26 22:53:32,2021-12-02T18:42:38.445Z,0.242248


### Generate partitioned orders data by month

In [52]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

print(f'Total Orders Count = {orders_df.shape[0]}') 
partitions_path = '../data/partitions'
start_date_str = '2020-01-01 00:00:00'
end_date_str = '2021-06-01 00:00:01'
date_format = '%Y-%m-%d %H:%M:%S'
start_date = datetime.strptime(start_date_str, date_format)
print(f'start_date = {start_date}')
end_date = datetime.strptime(end_date_str, date_format)
print(f'end_date = {end_date}')
a_month = relativedelta(months=1)
print(f'a_month = {a_month}')
current_start_date = start_date
print(f'current_start_date = {current_start_date}')
current_end_date = start_date + a_month
print(f'current_end_date = {current_end_date}')
print(f'----')
if not os.path.exists(partitions_path):
    os.makedirs(partitions_path)

while current_end_date <= end_date:
    print(f'Dates between {current_start_date} and {current_end_date}')
    partitions_df = orders_df[orders_df['purchased_on'].between(current_start_date, current_end_date)]
    partitions_df.drop('purchased_on', axis=1, inplace=True)
    partition = f'{current_start_date.strftime("%Y")}-{int(current_start_date.strftime("%m"))}'    
    current_partitions_path = f'{partitions_path}/{partition}'
    print(current_partitions_path)
    if not os.path.exists(current_partitions_path):
        os.makedirs(current_partitions_path)
    print(f'Partitions Orders Count = {partitions_df.shape[0]}')
    partitions_df.to_csv(f'{current_partitions_path}/partition.csv', index=False)
    partitions_df.iloc[0:0]
    current_start_date = current_end_date
    current_end_date = current_start_date + a_month
    print(f'----')

Total Orders Count = 100000
start_date = 2020-01-01 00:00:00
end_date = 2021-06-01 00:00:01
a_month = relativedelta(months=+1)
current_start_date = 2020-01-01 00:00:00
current_end_date = 2020-02-01 00:00:00
----
Dates between 2020-01-01 00:00:00 and 2020-02-01 00:00:00
../data/partitions/2020-1
Partitions Orders Count = 5940
----
Dates between 2020-02-01 00:00:00 and 2020-03-01 00:00:00
../data/partitions/2020-2
Partitions Orders Count = 5488


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


----
Dates between 2020-03-01 00:00:00 and 2020-04-01 00:00:00
../data/partitions/2020-3
Partitions Orders Count = 5957
----
Dates between 2020-04-01 00:00:00 and 2020-05-01 00:00:00
../data/partitions/2020-4
Partitions Orders Count = 5885
----
Dates between 2020-05-01 00:00:00 and 2020-06-01 00:00:00
../data/partitions/2020-5
Partitions Orders Count = 5988
----
Dates between 2020-06-01 00:00:00 and 2020-07-01 00:00:00
../data/partitions/2020-6
Partitions Orders Count = 5755
----
Dates between 2020-07-01 00:00:00 and 2020-08-01 00:00:00
../data/partitions/2020-7
Partitions Orders Count = 6127
----
Dates between 2020-08-01 00:00:00 and 2020-09-01 00:00:00
../data/partitions/2020-8
Partitions Orders Count = 6110
----
Dates between 2020-09-01 00:00:00 and 2020-10-01 00:00:00
../data/partitions/2020-9
Partitions Orders Count = 5767
----
Dates between 2020-10-01 00:00:00 and 2020-11-01 00:00:00
../data/partitions/2020-10
Partitions Orders Count = 6053
----
Dates between 2020-11-01 00:00:00 

In [53]:
orders_df.drop('purchased_on', axis=1, inplace=True)

In [54]:
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C5731,P16,0.913465,1,2021-12-02T18:42:38.444Z,0.122093
1,O2,C3541,P12802,0.663168,1,2021-12-02T18:42:38.444Z,0.903101
2,O3,C7402,P8320,0.629604,1,2021-12-02T18:42:38.444Z,0.054264
3,O4,C7356,P5165,0.618911,0,2021-12-02T18:42:38.445Z,0.343023
4,O5,C5806,P12940,0.053168,1,2021-12-02T18:42:38.445Z,0.242248


In [55]:
orders_df.to_csv('../data/transformed/orders.csv', index=False)